## Packages

In [1]:
import tensorflow as tf
import mlflow
import os
from pathlib import Path 
from time import strftime 
import time
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd

2026-01-04 15:12:53.258032: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-04 15:12:53.341748: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-04 15:12:55.391033: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


## Data

### Load data

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full,y_train_full), (X_test, y_test) = fashion_mnist

In [3]:
class_names = ["Tshirt", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle"]

In [4]:
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [5]:
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255. 
X_train[0].shape

(28, 28)

### Store img test

In [7]:
path = '../mnist-fashion-classification/public/images/'
for num,class_name in enumerate(class_names):
    index = [i for i,value in enumerate(y_test) if value==num]
    for i,lab in enumerate(index[:50]):
        os.makedirs( path + class_name, exist_ok=True)
        img_array = (X_test[lab] * 255).astype(np.uint8)
        Image.fromarray(img_array).save( path + class_name + '/' + str(i) + '.jpg')

## Models

In [12]:
tf.random.set_seed(2025)

#### Simple model

In [8]:
exp_pr = {
    "input_shape": (28, 28, 1),
    "hidden_units": [300, 200, 100],
    "activation": "relu",
    "kernel_initializer": "he_normal",
    "num_classes": 10,
    "optimizer": "adam",
    "lr": 0.01
}

In [7]:
def build_model(config: dict) -> tf.keras.Model:
    model = tf.keras.Sequential()

    # Input
    model.add(tf.keras.Input(shape=config["input_shape"]))
    model.add(tf.keras.layers.Flatten())

    # Hidden layers (dynamic)
    for units in config["hidden_units"]:
        model.add(
            tf.keras.layers.Dense(
                units,
                activation=config["activation"],
                kernel_initializer=config["kernel_initializer"]
            )
        )

    # Output layer
    model.add(
        tf.keras.layers.Dense(
            config["num_classes"],
            activation="softmax"
        )
    )

    # Optimizer
    optimizer_name = config["optimizer"].lower()
    lr = config["lr"]

    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer_name == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
    elif optimizer_name == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

    # Compile
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model



In [9]:
model = build_model(exp_pr)

2026-01-04 15:24:51.672595: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 200)            │        60,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 316,810 (1.21 MB)

 Trainable params: 316,810 (1.21 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(28, 28,1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.Dense(200, activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.Dense(100, activation='relu', kernel_initializer='he_normal'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

2026-01-03 16:34:57.767463: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


#### Model with Batch normalization

In [8]:
model_batch = tf.keras.Sequential()
model_batch.add(tf.keras.Input(shape=(28, 28, 1)))
model_batch.add(tf.keras.layers.Flatten())

# Dense layer 1
model_batch.add(tf.keras.layers.Dense(300, kernel_initializer='he_normal', use_bias=False))
model_batch.add(tf.keras.layers.BatchNormalization())
model_batch.add(tf.keras.layers.Activation('relu'))

# Dense layer 2
model_batch.add(tf.keras.layers.Dense(200, kernel_initializer='he_normal', use_bias=False))
model_batch.add(tf.keras.layers.BatchNormalization())
model_batch.add(tf.keras.layers.Activation('relu'))

# Dense layer 3
model_batch.add(tf.keras.layers.Dense(100, kernel_initializer='he_normal', use_bias=False))
model_batch.add(tf.keras.layers.BatchNormalization())
model_batch.add(tf.keras.layers.Activation('relu'))

# Output layer
model_batch.add(tf.keras.layers.Dense(10, activation='softmax'))

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model_batch.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

#### Model with batch and dropout

In [9]:
model_b_r = tf.keras.Sequential()
model_b_r.add(tf.keras.layers.InputLayer(input_shape=(28, 28,1)))
model_b_r.add(tf.keras.layers.Flatten())  # <-- Add this line!


# Dense layer 1
model_b_r.add(tf.keras.layers.Dense(300, kernel_initializer='he_normal', use_bias=False))
model_b_r.add(tf.keras.layers.BatchNormalization())
model_b_r.add(tf.keras.layers.Activation('relu'))
model_b_r.add(tf.keras.layers.Dropout(0.2))

# Dense layer 2
model_b_r.add(tf.keras.layers.Dense(200, kernel_initializer='he_normal', use_bias=False))
model_b_r.add(tf.keras.layers.BatchNormalization())
model_b_r.add(tf.keras.layers.Activation('relu'))
model_b_r.add(tf.keras.layers.Dropout(0.2))

# Dense layer 3
model_b_r.add(tf.keras.layers.Dense(100, kernel_initializer='he_normal', use_bias=False))
model_b_r.add(tf.keras.layers.BatchNormalization())
model_b_r.add(tf.keras.layers.Activation('relu'))
model_b_r.add(tf.keras.layers.Dropout(0.2))

# Output layer (no regularization here usually)
model_b_r.add(tf.keras.layers.Dense(10, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model_b_r.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

#### Model with batch, droutout and schedule

In [10]:
model_b_r_s = tf.keras.Sequential()
model_b_r_s.add(tf.keras.layers.InputLayer(input_shape=(28, 28,1)))
model_b_r_s.add(tf.keras.layers.Flatten())  # <-- Add this line!


# Dense layer 1
model_b_r_s.add(tf.keras.layers.Dense(300, kernel_initializer='he_normal', use_bias=False))
model_b_r_s.add(tf.keras.layers.BatchNormalization())
model_b_r_s.add(tf.keras.layers.Activation('relu'))
model_b_r_s.add(tf.keras.layers.Dropout(0.2))

# Dense layer 2
model_b_r_s.add(tf.keras.layers.Dense(200, kernel_initializer='he_normal', use_bias=False))
model_b_r_s.add(tf.keras.layers.BatchNormalization())
model_b_r_s.add(tf.keras.layers.Activation('relu'))
model_b_r_s.add(tf.keras.layers.Dropout(0.2))

# Dense layer 3
model_b_r_s.add(tf.keras.layers.Dense(100, kernel_initializer='he_normal', use_bias=False))
model_b_r_s.add(tf.keras.layers.BatchNormalization())
model_b_r_s.add(tf.keras.layers.Activation('relu'))
model_b_r_s.add(tf.keras.layers.Dropout(0.2))

# Output layer (no regularization here usually)
model_b_r_s.add(tf.keras.layers.Dense(10, activation='softmax'))

# Optimizer
initial_lr = 0.01

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_lr,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model_b_r_s.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train models

### Set up Tensorboard

In [7]:
def get_run_logdir(root_logdir="my_logs"):
	return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S") 

run_logdir = get_run_logdir()  # e.g., my_logs/run_2022_08_01_17_25_59 

In [8]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=run_logdir,
    histogram_freq=1
)

run the server tensorboard --logdir ./my_logs

## Set up MLFlow

First run the mlflow server

In [12]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("my-first-experiment")

2026/01/04 15:28:50 INFO mlflow.tracking.fluent: Experiment with name 'my-first-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1767558530543, experiment_id='1', last_update_time=1767558530543, lifecycle_stage='active', name='my-first-experiment', tags={}>

In [13]:
# Print connection information
print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Active Experiment: {mlflow.get_experiment_by_name('my-first-experiment')}")

# Test logging
with mlflow.start_run():
    mlflow.log_param("test_param", "test_value")
    print("✓ Successfully connected to MLflow!")

MLflow Tracking URI: http://localhost:5000
Active Experiment: <Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1767558530543, experiment_id='1', last_update_time=1767558530543, lifecycle_stage='active', name='my-first-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>
✓ Successfully connected to MLflow!
🏃 View run salty-dolphin-916 at: http://localhost:5000/#/experiments/1/runs/46a45aeee9c54500952706c4c480bd7f
🧪 View experiment at: http://localhost:5000/#/experiments/1


## Logging models

In [17]:
model_params = {
    "input_shape": (28, 28, 1),
    "hidden_units": [300, 200, 100],
    "activation": "relu",
    "kernel_initializer": "he_normal",
    "num_classes": 10,
    "optimizer": "adam",
    "lr": 0.01
}


In [18]:
training_params = {
    "epochs": 3,
    "batch_size": 64,
    "experiment_name": "model"
}

In [20]:
model = build_model(model_params)

In [23]:
runs_info = []

experiments = {
    "name": "model",
    "epochs": 3,
    "batch_size": 8
}

with mlflow.start_run(run_name=experiments["name"]):

    # ---- log training params ----
    mlflow.log_param("epochs", experiments["epochs"])
    mlflow.log_param("batch_size", experiments["batch_size"])

    # ---- TRAIN (no history needed) ----
    model.fit(
        X_train,
        y_train,
        epochs=experiments["epochs"],
        batch_size=experiments["batch_size"],
        validation_data=(X_valid, y_valid),
        callbacks=[mlflow.tensorflow.MlflowCallback()],
        verbose=False
    )

    # ---- EVALUATION using compiled metrics ----
    val_results = model.evaluate(X_valid, y_valid, verbose=False)
    test_results = model.evaluate(X_test, y_test, verbose=False)

    # ---- map metric names to values ----
    val_metrics = dict(zip(
        [f"val_{name}" for name in model.metrics_names],
        val_results
    ))

    test_metrics = dict(zip(
        [f"test_{name}" for name in model.metrics_names],
        test_results
    ))

    # ---- log ALL metrics dynamically ----
    mlflow.log_metrics({
        **val_metrics,
        **test_metrics
    })

    # ---- log model artifact (NOT register) ----
    mlflow.tensorflow.log_model(
        model,
        artifact_path="model"
    )

    # ---- run info ----
    run_id = mlflow.active_run().info.run_id
    model_uri = f"runs:/{run_id}/model"

    print(run_id)
    print(model_uri)

2026-01-04 15:57:41.613911: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 172480000 exceeds 10% of free system memory.


2026-01-04 15:58:44.028884: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 15680000 exceeds 10% of free system memory.
2026/01/04 15:58:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/04 15:58:45 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


c6293dd9e003426e8d45a20b61e0fbe8
runs:/c6293dd9e003426e8d45a20b61e0fbe8/model
🏃 View run model at: http://localhost:5000/#/experiments/1/runs/c6293dd9e003426e8d45a20b61e0fbe8
🧪 View experiment at: http://localhost:5000/#/experiments/1


### Load model

In [52]:
# Load model by alias (best practice)
loaded_model = mlflow.tensorflow.load_model(
    model_uri="models:/first_model_register@first_champion"
)

In [53]:
results = loaded_model.evaluate(X_test, y_test, verbose=False)

In [54]:
results

[0.4404793083667755, 0.845300018787384]

In [27]:
predictions = loaded_model.predict(X_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [55]:
np.sum(y_test==np.argmax(predictions, axis=1))

np.int64(8453)

In [ ]:
y_test

(10000,)

In [57]:
(8453/10000)

0.8453

## Tensorboard

In [8]:
import os
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

def load_event_tags(log_dir):
    ea = EventAccumulator(log_dir)
    ea.Reload()
    return ea

def extract_values(events):
    steps = [e.step for e in events]
    values = [e.value for e in events]
    return steps, values

# Paths to your logs
train_log = "/home/angel/Documents/MNIST-fashion-classification/notebooks/my_logs/run_2025_06_18_09_49_17/train"
val_log   = "/home/angel/Documents/MNIST-fashion-classification/notebooks/my_logs/run_2025_06_18_09_49_17/validation"

# Load event accumulators
train_ea = load_event_tags(train_log)
val_ea   = load_event_tags(val_log)

# Show tags from TensorBoard
print("📦 Train log tags:")
for k, v in train_ea.Tags().items():
    print(f"{k}: {v}")

print("\n📦 Validation log tags:")
for k, v in val_ea.Tags().items():
    print(f"{k}: {v}")

# Check for scalar tags
train_scalar_tags = train_ea.Tags().get('scalars', [])
val_scalar_tags = val_ea.Tags().get('scalars', [])

common_tags = list(set(train_scalar_tags) & set(val_scalar_tags))

if not common_tags:
    print("\n❌ No scalar tags found in both logs to plot.")
else:
    print(f"\n✅ Common scalar tags found: {common_tags}")

    # Plotting
    fig, axes = plt.subplots(1, len(common_tags), figsize=(6 * len(common_tags), 5))

    # Ensure axes is always iterable
    if len(common_tags) == 1:
        axes = [axes]

    for i, tag in enumerate(common_tags):
        train_events = train_ea.Scalars(tag)
        val_events = val_ea.Scalars(tag)

        train_steps, train_values = extract_values(train_events)
        val_steps, val_values = extract_values(val_events)

        axes[i].plot(train_steps, train_values, label=f"Train {tag}")
        axes[i].plot(val_steps, val_values, label=f"Val {tag}")
        axes[i].set_title(tag)
        axes[i].set_xlabel("Step")
        axes[i].set_ylabel(tag)
        axes[i].legend()

    plt.tight_layout()
    plt.show()


📦 Train log tags:
images: []
audio: []
histograms: []
scalars: []
distributions: []
tensors: ['keras', 'epoch_loss', 'epoch_accuracy', 'epoch_learning_rate']
graph: False
meta_graph: False
run_metadata: []

📦 Validation log tags:
images: []
audio: []
histograms: []
scalars: []
distributions: []
tensors: ['evaluation_loss_vs_iterations', 'evaluation_accuracy_vs_iterations', 'epoch_loss', 'epoch_accuracy']
graph: False
meta_graph: False
run_metadata: []

❌ No scalar tags found in both logs to plot.


In [4]:
import os
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tensorflow.python.framework import tensor_util

# Paths to log directories
train_dir = "/home/angel/Documents/MNIST-fashion-classification/notebooks/my_logs/run_2025_06_18_09_49_17/train"
val_dir   = "/home/angel/Documents/MNIST-fashion-classification/notebooks/my_logs/run_2025_06_18_09_49_17/validation"

# Define model labels manually in the order they were trained
model_names = [
    "Baseline",
    "BatchNorm",
    "Dropout + BatchNorm",
    "Dropout + BN + LR Scheduler"
]

def get_last_tensor_value(events):
    if not events:
        return None
    tensor_proto = events[-1].tensor_proto
    return float(tensor_util.make_ndarray(tensor_proto).squeeze())

def extract_metrics_from_file(path, is_train=True):
    ea = EventAccumulator(path)
    ea.Reload()
    tags = ea.Tags().get("tensors", [])

    def get(tag):
        try:
            if tag in tags:
                return get_last_tensor_value(ea.Tensors(tag))
            return None
        except:
            return None

    if is_train:
        return {
            "Train Loss": get("epoch_loss"),
            "Train Accuracy": get("epoch_accuracy"),
            "Final LR": get("epoch_learning_rate")
        }
    else:
        return {
            "Val Loss": get("evaluation_loss_vs_iterations"),
            "Val Accuracy": get("evaluation_accuracy_vs_iterations")
        }

# Get all event files
train_files = sorted([
    os.path.join(train_dir, f)
    for f in os.listdir(train_dir)
    if f.startswith("events.out")
])

val_files = sorted([
    os.path.join(val_dir, f)
    for f in os.listdir(val_dir)
    if f.startswith("events.out")
])

# Combine metrics
data = []
for i in range(min(len(train_files), len(val_files))):
    row = {"Model": model_names[i] if i < len(model_names) else f"Model {i+1}"}
    row.update(extract_metrics_from_file(train_files[i], is_train=True))
    row.update(extract_metrics_from_file(val_files[i], is_train=False))
    data.append(row)

# Create table
df = pd.DataFrame(data)
df = df[["Model", "Train Loss", "Train Accuracy", "Val Loss", "Val Accuracy", "Final LR"]]

print("\n📊 Model Comparison Table:\n")
print(df)



📊 Model Comparison Table:

                         Model Train Loss Train Accuracy Val Loss  \
0                     Baseline       None           None     None   
1                    BatchNorm       None           None     None   
2          Dropout + BatchNorm       None           None     None   
3  Dropout + BN + LR Scheduler       None           None     None   

  Val Accuracy Final LR  
0         None     None  
1         None     None  
2         None     None  
3         None     None  


In [9]:
import os
import tensorflow as tf
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

# Rutas a los logs
train_dir = "/home/angel/Documents/MNIST-fashion-classification/notebooks/my_logs/run_2025_06_18_09_49_17/train"
val_dir = "/home/angel/Documents/MNIST-fashion-classification/notebooks/my_logs/run_2025_06_18_09_49_17/validation"

# Nombres de modelos en orden
model_names = [
    "Baseline",
    "BatchNorm",
    "Dropout + BatchNorm",
    "Dropout + BN + LR Scheduler"
]

# Leer el último valor de un tensor
def get_last_tensor_value(events):
    if not events:
        return None
    return float(tf.make_ndarray(events[-1].tensor_proto).squeeze())

# Extraer métricas de un archivo
def extract_metrics(file_path, keys):
    ea = EventAccumulator(file_path)
    ea.Reload()
    tensors = ea.Tags().get("tensors", [])
    values = {}
    for key in keys:
        if key in tensors:
            events = ea.Tensors(key)
            values[key] = get_last_tensor_value(events)
        else:
            values[key] = None
    return values

# Obtener archivos
train_files = sorted([
    os.path.join(train_dir, f) for f in os.listdir(train_dir)
    if f.startswith("events.out")
])

val_files = sorted([
    os.path.join(val_dir, f) for f in os.listdir(val_dir)
    if f.startswith("events.out")
])

# Armar tabla
data = []
for i in range(min(len(train_files), len(val_files))):
    row = {"Model": model_names[i] if i < len(model_names) else f"Model {i+1}"}
    train_metrics = extract_metrics(train_files[i], ["epoch_loss", "epoch_accuracy"])
    val_metrics = extract_metrics(val_files[i], ["epoch_loss", "epoch_accuracy"])

    row["Train Loss"] = round(train_metrics["epoch_loss"], 4) if train_metrics["epoch_loss"] is not None else None
    row["Train Accuracy"] = round(train_metrics["epoch_accuracy"], 4) if train_metrics["epoch_accuracy"] is not None else None
    row["Val Loss"] = round(val_metrics["epoch_loss"], 4) if val_metrics["epoch_loss"] is not None else None
    row["Val Accuracy"] = round(val_metrics["epoch_accuracy"], 4) if val_metrics["epoch_accuracy"] is not None else None

    data.append(row)

# Mostrar tabla
df = pd.DataFrame(data)
print("\n📊 Model Comparison Table:\n")
print(df)



📊 Model Comparison Table:

                         Model  Train Loss  Train Accuracy  Val Loss  \
0                     Baseline      0.2280          0.9237    1.3741   
1                    BatchNorm      0.0179          0.9943    0.8862   
2          Dropout + BatchNorm      0.0576          0.9801    0.5158   
3  Dropout + BN + LR Scheduler      0.0303          0.9892    0.5414   

   Val Accuracy  
0        0.8652  
1        0.8894  
2        0.8992  
3        0.9052  
